In [32]:
import pandas as pd
df = pd.read_csv('Data/Train.csv')
df.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png


In [35]:
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    #vgg16 use images of size (224, 244)
    img = img.resize((224, 224))

    return img
X = [load_and_preprocess_image(path) for path in df['Path']]

In [36]:
y = df['ClassId'].values

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import VGG16
import numpy as np

In [43]:
# Step 4: Preprocess Images for VGG16
#X_train = np.array([preprocess_input(np.array(img)) for img in X_train])
#X_val = np.array([preprocess_input(np.array(img)) for img in X_val])

In [45]:
base_model = VGG16(weights = 'imagenet',
                   include_top = False,  # drop the top layer
                   input_shape = (224,224, 3)) 

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 22s 0us/step


In [48]:
num_class = df['ClassId'].max()
num_class

42

In [52]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [51]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation = 'relu')(x)
predictions = Dense(42, activation = 'sigmoid')(x)

In [53]:
model = Model(inputs = base_model.input, outputs= predictions)

In [64]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer= optimizer,
                        loss = 'binary_crossentropy',
                        metrics =['accuracy'])

In [61]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [62]:
# Print shapes and types for debugging
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_train type:", type(X_train))
print("y_train type:", type(y_train))

X_train shape: (31367, 224, 224, 3)
y_train shape: (31367,)
X_train type: <class 'numpy.ndarray'>
y_train type: <class 'numpy.ndarray'>


In [65]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val)
)

MemoryError: Unable to allocate 4.40 GiB for an array with shape (31367, 224, 224, 3) and data type uint8

In [ ]:
import matplotlib as plt
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()